## Generación de imágenes sintéticas

En este notebook usaremos un modelo base de StableDifusion para la generación de imagenes de emociones, las cuáles usaremos posteriormente para realizar el entrenamiento de un modelo de clasificación de emociones

In [1]:
### install requirements 

!pip install --upgrade diffusers[torch]
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 kB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.5/435.5 kB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 136.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 152.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 155.3 MB/s eta 0:00:00


In [2]:
### create image generation pipeline ###

from diffusers import DiffusionPipeline
import torch


pipeline = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16)
pipeline.to("cuda")


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

scheduler/scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

...

tokenizer/tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.30.0",
  "_name_or_path": "runwayml/stable-diffusion-v1-5",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "image_encoder": [
    null,
    null
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [ ]:
### generate images ###

import random
import os

import matplotlib.pyplot as plt


os.makedirs('./content/faces/happy', exist_ok=True)
os.makedirs('./content/faces/sad', exist_ok=True)
os.makedirs('./content/faces/angry', exist_ok=True)
os.makedirs('./content/faces/surprised', exist_ok=True)


ethnicities = ['a latino', 'a white', 'a black', 'a middle eastern', 'an indian', 'an asian']

genders = ['male', 'female']

emotion_prompts = {'happy': 'smiling',
                   'sad': 'frowning, sad face expression, crying',
                   'surprised': 'surprised, opened mouth, raised eyebrows',
                   'angry': 'angry'}


for j in range(250):

  for emotion in emotion_prompts.keys():

    emotion_prompt = emotion_prompts[emotion]

    ethnicity = random.choice(ethnicities)
    gender = random.choice(genders)

    # print(emotion, ethnicity, gender)

    prompt = 'Medium-shot portrait of {} {}, {}, front view, looking at the camera, color photography, '.format(ethnicity, gender, emotion_prompt) + \
            'photorealistic, hyperrealistic, realistic, incredibly detailed, crisp focus, digital art, depth of field, 50mm, 8k'
    negative_prompt = '3d, cartoon, anime, sketches, (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ' + \
                      '((grayscale)) Low Quality, Worst Quality, plastic, fake, disfigured, deformed, blurry, bad anatomy, blurred, watermark, grainy, signature'

    img = pipeline(prompt, negative_prompt=negative_prompt).images[0]

    img.save('./content/faces/{}/{}.png'.format(emotion, str(j).zfill(4)))

    # plt.imshow(img)
    # plt.show()


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Si estamos usando Google Colab y queremos ver las imagenes:

In [5]:
!zip -r faces.zip ./content/faces

  adding: content/faces/ (stored 0%)
  adding: content/faces/angry/ (stored 0%)
  adding: content/faces/angry/0093.png (deflated 0%)
  adding: content/faces/angry/0134.png (deflated 0%)
  adding: content/faces/angry/0165.png (deflated 0%)
  adding: content/faces/angry/0249.png (deflated 0%)
  adding: content/faces/angry/0193.png (deflated 0%)
  adding: content/faces/angry/0119.png (deflated 0%)
  adding: content/faces/angry/0155.png (deflated 0%)
  adding: content/faces/angry/0153.png (deflated 0%)
  adding: content/faces/angry/0075.png (deflated 0%)
  adding: content/faces/angry/0127.png (deflated 0%)
  adding: content/faces/angry/0216.png (deflated 0%)
  adding: content/faces/angry/0090.png (deflated 0%)
  adding: content/faces/angry/0069.png (deflated 0%)
  adding: content/faces/angry/0019.png (deflated 0%)
  adding: content/faces/angry/0094.png (deflated 0%)
  adding: content/faces/angry/0061.png (deflated 0%)
  adding: content/faces/angry/0176.png (deflated 0%)
  adding: content/f

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
!scp '/content/faces.zip' '/content/gdrive/My Drive/SyntheticDatasetFaceGenerationStableDiffusion/faces.zip'